In [ ]:
import time
import numpy as np
import requests
import json

from datetime import datetime, timedelta

In [ ]:
base = 'https://api.spoonacular.com'
api_key = 'apiKey=556d5c003785468ab5aa696a128a3d3a'

In [86]:
def get_mealplan_templates():
    r = requests.get(base + '/mealplanner/public-templates?' + api_key)
    return json.loads(r.content.decode())

    
def connect_user():
    r = requests.post(base + '/users/connect?' + api_key, data='{}')
    return json.loads(r.content.decode())


def add_to_user_plan(user, mealplan_template_id):
    date = round(time.time() + 100)
    json_data = json.dumps({
        'mealPlanTemplateId': mealplan_template_id, 'startDate': date
    })
    r = requests.post(
        base + f"/mealplanner/{user['username']}/items?{api_key}&hash={user['hash']}",
        data=json_data)
    return json.loads(r.content.decode())


def generate_shopping_list(user):
    date_today = datetime.strftime(datetime.now(), '%Y-%m-%d')
    date_in_a_week = datetime.strftime(datetime.now() + timedelta(7), '%Y-%m-%d')
    r = requests.post(
        base + '/mealplanner/' + user['username'] + '/shopping-list/' + date_today 
        + '/' + date_in_a_week + '?' + api_key + '&hash=' + user['hash'])
    return json.loads(r.content.decode())

In [ ]:
templates = get_mealplan_templates()
i = 9
for template in np.random.choice(templates['templates'], size=10):
    user = connect_user()
    id_ = template['id']
    add_to_user_plan(user, id_)
    with open(f'../shopping_lists/shoppingList{i}.json', 'w+') as f:
        json.dump(generate_shopping_list(user), f, indent=2, ensure_ascii=False)
    i += 1